## 10 Step Project (Image classification Cat vs Dog)
#### 1. Import libraries
#### 2. Download data
#### 3. Data cleaning
#### 4. Data preprocessing and converting to tensros
#### 5. Building model
#### 6. Loss and optimizer functions
#### 7. Train the model
#### 8. Test the model
#### 9. Save the model
#### 10. Load saved model

## Step1: Import all libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os
from PIL import Image

## Step2: Download data

In [ ]:
!conda install -y gdown # or use !pip3 install -y gdown if using conda 
!gdown https://drive.google.com/uc?id=1TWjI7ucryVPGMpUMq8cdigYrtKQtc54t # downloading cat and dog images
!unzip kagglecatsanddogs_3367a.zip 

## Step3: Clean data and remove non image files

In [ ]:
# cleaning data

path = 'PetImages'

for folder in os.listdir(path): # Get all folder names --> ['Cat', 'Dog']
    for img_file in os.listdir(os.path.join(path, folder)): # Loop each folder to get all image files
        img_file = os.path.join(path, folder, img_file) # creating full path for each image file
        
        try:
            img = Image.open(img_file)
            if img.mode != 'RGB':
                os.remove(img_file) # removing gray scale images 
        except:
            os.remove(img_file) # removing file type None images


## Step4: Data preprocessing and convert to tensor format


In [ ]:
# pre-process data

transform = transforms.Compose([
                                transforms.Resize(255),  # resize img to 255px square image
                                transforms.CenterCrop(224), # then center crop by 224px
                                transforms.ToTensor(), # convert into tensor format
                                transforms.Normalize([0.5], [0.5]) # normalize tensors.
                               ]) 

dataset = datasets.ImageFolder('PetImages', transform=transform)

dataset_len = len(dataset) # get total number of images in all folders.

train_len, test_len = dataset_len-6000, 6000 # Keep 6000 images for test_set and others are for training
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len]) # split train and test dataset
batch_size = 200

# train and test dataloader
train_set = DataLoader(dataset=train_set, shuffle=True, batch_size=batch_size)
test_set = DataLoader(dataset=test_set, shuffle=True, batch_size=batch_size)

# if cuda available then use device as cuda else use cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Using device: ',device)

## Step5: Build Model


In [ ]:
# CNN model 

class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    
    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout(p=0.2)
    
    self.conv1 = nn.Conv2d( in_channels=3,  out_channels=6,  kernel_size=4)
    self.conv2 = nn.Conv2d( in_channels=6,  out_channels=12, kernel_size=4)
    self.conv3 = nn.Conv2d( in_channels=12, out_channels=14, kernel_size=4)
    self.conv4 = nn.Conv2d( in_channels=14, out_channels=16, kernel_size=4)
    self.conv5 = nn.Conv2d( in_channels=16, out_channels=20, kernel_size=4)
    
    self.fc1 = nn.Linear( in_features= 20*4*4, out_features=250 ) 
    self.fc2 = nn.Linear( in_features=250,     out_features=200 )
    self.fc3 = nn.Linear( in_features=200,     out_features=50  )
    self.fc4 = nn.Linear( in_features=50,      out_features=10  )
    self.fc5 = nn.Linear( in_features=10,      out_features=2   )
  
  
  def forward(self, x): # sahpe of x is [batch_size, channel, height, width]
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = self.pool(F.relu(self.conv4(x)))
    x = self.pool(F.relu(self.conv5(x)))

    x = x.reshape(-1, 20*4*4)
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    x = self.dropout(F.relu(self.fc4(x)))
    x = self.fc5(x)
    return x


net = Model().to(device)

print(net)

## step6: Define Loss and optimizer functions

In [ ]:
# optimizer and loss functions
criterion = nn.CrossEntropyLoss() # crossentropy loss for classification problem
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5) # lr and weight_decay are hyper parameters
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 

## Step7: Train Data

In [ ]:
# train model

net.train()

for epoch in range(15):
  total_correct = 0.0
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_set):
    inputs, labels = inputs.to(device), labels.to(device)
    output = net(inputs)
    output_idx = torch.argmax(output, dim=1)
    total_correct += (labels == output_idx).sum().item()
    optimizer.zero_grad()
    loss = criterion(output, labels)
    running_loss += loss.item() * inputs.size(0)
    loss.backward()
    optimizer.step()
    
  print(f'Epoch: {epoch}  Loss: {running_loss/train_len}   Accuracy:{(total_correct/train_len)*100}%')

print('Finished training')

## Step8: Test Data

In [ ]:
# Test our model

with torch.no_grad(): 
  net.eval()
  total_loss = 0.0
  total_correct = 0.0
  
  for inputs, labels in test_set:
    labels = labels.to(device)
    outputs = net(inputs.to(device))
    loss = criterion(outputs, labels)
    total_loss += loss.item() * inputs.size(0)
    output_idx = torch.argmax(outputs, dim=1)
    total_correct += sum(labels==output_idx)
    
  print(f'Accuracy : {(total_correct/test_len)*100}%  Loss: {total_loss/ test_len}')

## Step9: Save trained model

In [ ]:
# Save the model
torch.save(net.state_dict(), 'cat_vs_dog.pt')

## Step10: Load saved model and test it on test data

In [ ]:

# Test from saved .pt model
with torch.no_grad():
  model = Model().to(device)
  model.load_state_dict(torch.load('cat_vs_dog.pt'))
  model.eval()

  total_correct = 0.0

  for inputs, labels in test_set:
    labels = labels.to(device)
    outputs = model(inputs.to(device))
    output_idx = torch.argmax(outputs, dim=1)
    total_correct += sum(labels==output_idx)
  print(f'Accuracy : {(total_correct/test_len)*100}% ')
    